In [1]:
import tensorflow as tf

import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm 
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/project/source_price.csv")


In [6]:
df

,date,wsj_mean_compound,cnbc_mean_compound,fortune_mean_compound,reuters_mean_compound,Adj Close
0,2017/12/7,0.296000,-0.136600,0.000000,0.000000,2636.979980
1,2017/12/8,0.000000,0.000000,-0.242300,0.000000,2651.500000
2,2017/12/11,0.000000,0.000000,0.000000,0.000000,2659.989990
3,2017/12/12,0.000000,0.000000,0.000000,0.000000,2664.110107
4,2017/12/13,0.000000,0.000000,0.000000,0.000000,2662.850098
...,...,...,...,...,...,...
116,2018/5/25,0.030290,0.047433,0.011550,-0.025190,2721.330078
117,2018/5/29,-0.052796,0.070442,-0.025721,-0.035568,2689.860107
118,2018/5/30,-0.017367,0.038119,-0.076965,-0.063177,2724.010010
119,2018/5/31,-0.018636,0.057371,-0.064138,-0.025489,2705.270020


In [4]:
wsj_var=np.var(df.wsj_mean_compound)
cnbc_var=np.var(df.cnbc_mean_compound)
fortune_var=np.var(df.fortune_mean_compound)
reuters_var=np.var(df.reuters_mean_compound)

In [5]:
mu=0

noise=0.1


In [6]:
sigma_wsj=noise*wsj_var
sigma_cnbc=noise*cnbc_var
sigma_fortune=noise*fortune_var
sigma_reuters=noise*reuters_var

In [7]:
n=df.shape[0]

df_noise=pd.DataFrame()

In [8]:
df_noise['wsj_noise']=df['wsj_mean_compound']
df_noise['cnbc_noise']=df['cnbc_mean_compound']
df_noise['fortune_noise']=df['fortune_mean_compound']
df_noise['reuters_noise']=df['reuters_mean_compound']

In [9]:

for i in range(0,n):
    df_noise['wsj_noise'][i]+=np.random.normal(mu,sigma_wsj)
    df_noise['cnbc_noise'][i]+=np.random.normal(mu,sigma_cnbc)
    df_noise['fortune_noise'][i]+=np.random.normal(mu,sigma_fortune)
    df_noise['reuters_noise'][i]+=np.random.normal(mu,sigma_reuters)

In [10]:
df_noise.to_csv("/content/drive/MyDrive/project/source_price_noise0.csv")

In [11]:
dfn=pd.read_csv("/content/drive/MyDrive/project/source_price_noise0.csv",index_col=0)  


In [12]:
df_1n=pd.DataFrame()
df_1n['wsj']=dfn['wsj_noise']
df_1n['cnbc']=df['cnbc_mean_compound']
df_1n['fortune']=df['fortune_mean_compound']
df_1n['reuters']=df['reuters_mean_compound']
df_1n['price']=df['Adj Close']

In [13]:
df_2n=pd.DataFrame()
df_2n['wsj']=df['wsj_mean_compound']
df_2n['cnbc']=dfn['cnbc_noise']
df_2n['fortune']=df['fortune_mean_compound']
df_2n['reuters']=df['reuters_mean_compound']
df_2n['price']=df['Adj Close']

df_3n=pd.DataFrame()
df_3n['wsj']=df['wsj_mean_compound']
df_3n['cnbc']=df['cnbc_mean_compound']
df_3n['fortune']=dfn['fortune_noise']
df_3n['reuters']=df['reuters_mean_compound']
df_3n['price']=df['Adj Close']

df_4n=pd.DataFrame()
df_4n['wsj']=df['wsj_mean_compound']
df_4n['cnbc']=df['cnbc_mean_compound']
df_4n['fortune']=df['fortune_mean_compound']
df_4n['reuters']=dfn['reuters_noise']
df_4n['price']=df['Adj Close']

df1=df_1n
df2=df_2n
df3=df_3n
df4=df_4n

In [31]:
split = (0.85)
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=100
prediction_len=1
dense_output=1
drop_out=0

In [15]:
i_split = int(len(df1) * split)

cols = ['price','wsj','cnbc','fortune','reuters']
data_train_1 = df1.get(cols).values[:i_split]
data_train_2 = df2.get(cols).values[:i_split]
data_train_3 = df3.get(cols).values[:i_split]
data_train_4 = df4.get(cols).values[:i_split]

len_train  = len(data_train_1)
len_train_windows = None

In [16]:
#get_train_data 
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_1[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
  
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

In [17]:
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train1 = data_windows[:, :-1]
y_train1 = data_windows[:, -1,[0]]
print('x_train1.shape',x_train1.shape)
print('y_train1.shape',y_train1.shape)

x_train1.shape (92, 9, 5)
y_train1.shape (92, 1)


In [18]:
#get_train_data 
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_2[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
  
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train2 = data_windows[:, :-1]
y_train2 = data_windows[:, -1,[0]]
print('x_train2.shape',x_train2.shape)
print('y_train2.shape',y_train2.shape)


x_train2.shape (92, 9, 5)
y_train2.shape (92, 1)


In [19]:
#get_train_data 
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_3[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
  
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train3 = data_windows[:, :-1]
y_train3 = data_windows[:, -1,[0]]
print('x_train3.shape',x_train3.shape)
print('y_train3.shape',y_train3.shape)

x_train3.shape (92, 9, 5)
y_train3.shape (92, 1)


In [20]:
#get_train_data 

data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_4[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
  
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train4 = data_windows[:, :-1]
y_train4 = data_windows[:, -1,[0]]
print('x_train4.shape',x_train4.shape)
print('y_train4.shape',y_train4.shape)

x_train4.shape (92, 9, 5)
y_train4.shape (92, 1)


In [21]:
##concat train window
type(x_train4)

numpy.ndarray

In [22]:
x_train_t=np.concatenate((x_train1,x_train2,x_train3,x_train4),axis=0)
print(x_train_t.shape)
x_train=x_train_t

y_train_t=np.concatenate((y_train1,y_train2,y_train3,y_train4),axis=0)
print(y_train_t.shape)
y_train=y_train_t

(368, 9, 5)
(368, 1)


In [23]:
dataframe= pd.read_csv("/content/drive/MyDrive/project/source_price.csv")
dataframe.columns=['date','wsj','cnbc','fortune','reuters','price']
cols = ['price','wsj','cnbc','fortune','reuters']
len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_test  = dataframe.get(cols).values[i_split:]

len_test   = len(data_test)
len_train_windows = None

print('data_test.shape',data_test.shape)

data_test.shape (19, 5)


In [24]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])

data_windows = np.array(data_windows).astype(float)

y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

y_test_ori.shape (9, 1)


In [25]:
y_test_ori


array([[2712.969971],
       [2733.01001 ],
       [2724.439941],
       [2733.290039],
       [2727.76001 ],
       [2721.330078],
       [2689.860107],
       [2724.01001 ],
       [2705.27002 ]])

In [26]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)

x_test.shape (9, 9, 5)
y_test.shape (9, 1)


In [32]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/100
4/4 [==============================] - 5s 27ms/step - loss: 0.4151
Epoch 2/100
4/4 [==============================] - 0s 25ms/step - loss: 0.2918
Epoch 3/100
4/4 [==============================] - 0s 25ms/step - loss: 0.2197
Epoch 4/100
4/4 [==============================] - 0s 29ms/step - loss: 0.2168
Epoch 5/100
4/4 [==============================] - 0s 25ms/step - loss: 0.1886
Epoch 6/100
4/4 [==============================] - 0s 23ms/step - loss: 0.1807
Epoch 7/100
4/4 [==============================] - 0s 28ms/step - loss: 0.1714
Epoch 8/100
4/4 [==============================] - 0s 25ms/step - loss: 0.1590
Epoch 9/100
4/4 [==============================] - 0s 27ms/step - loss: 0.1554
Epoch 10/100
4/4 [==============================] - 0s 25ms/step - loss: 0.1467
Epoch 11/100
4/4 [==============================] - 0s 25ms/step - loss: 0.1364
Epoch 12/100
4/4 [==============================] - 0s 25ms/step - loss: 0.1247
Epoch 13/100
4/4 [==============================]

In [33]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

In [34]:
#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

[2702.1449154568845, 2692.817959580754, 2712.298598234405, 2722.516507989063, 2722.966211181776, 2723.889213462677, 2701.5745506449425, 2697.799188951983, 2697.624951146796]


In [30]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])
    
squaredError = []
absError = []
for val in error:
    squaredError.append(val * val) 
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)

MSE [211.58897463]
accuracy [0.99560259]
mean_error_percent [0.00439741]
